### A project to practice SQL skills to answer business questions

In [80]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Context managers make our life easy by taking care of opening and closing the connection to the database, as well as making sure we don't accidentally make changes to the database if one of our queries has an error.[More here](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/)
Hence we will be using context managers here

In [81]:
def run_command(query):
    with sqlite3.connect('chinook.db') as conn:
        # tells SQLite to autocommit any changes
        conn.isolation_level = None
        return conn.execute(query)

In [82]:
def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)

In [83]:
def show_tables():
    query = "SELECT * FROM sqlite_master WHERE type ='table' OR type='view'"
    return run_query(query)

### Shows tables informaton in database

In [84]:
show_tables()

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


### Number of tracks sold in USA
 - in absolute numbers
 - in percentages

In [85]:
query='''
with invoice_usa_info as 
(
select inv_l.invoice_id,
inv_l.track_id
from invoice as inv 
LEFT JOIN invoice_line as inv_l 
on inv.invoice_id = inv_l.invoice_id 
where inv.billing_country='USA'
),
track_usa_info as
(
select artist.name as artist_name,
track.genre_id,
genre.name as genre_name,
track.track_id
from invoice_usa_info
LEFT JOIN track
on invoice_usa_info.track_id = track.track_id
LEFT JOIN album
on track.album_id = album.album_id
LEFT JOIN artist
on artist.artist_id = album.artist_id
LEFT JOIN genre
on track.genre_id = genre.genre_id
),
count_sold_artist_usa as
(
select artist_name,
genre_id,
COUNT(track_id) as count
from track_usa_info
GROUP BY artist_name 
),
most_sold_artist_usa as
(
select 
artist_name,
genre_id,
Max(count) as count_of_tracks
from count_sold_artist_usa
GROUP BY 2
),
sales_genres_usa as
(
select 
track_usa_info.genre_id,
track_usa_info.genre_name,
COUNT(track_usa_info.genre_id) as count, 
(cast(COUNT(track_usa_info.genre_id) as Float)/(Select Count(*) from track_usa_info))*100 as pecentage
from track_usa_info
GROUP BY track_usa_info.genre_id
ORDER BY 3 DESC
)
select 
sales_genres_usa.*,
most_sold_artist_usa.artist_name
from sales_genres_usa
LEFT JOIN most_sold_artist_usa
on sales_genres_usa.genre_id = most_sold_artist_usa.genre_id
'''
run_query(query)

,genre_id,genre_name,count,pecentage,artist_name
0,1,Rock,561,53.377735,Van Halen
1,4,Alternative & Punk,130,12.369172,Green Day
2,3,Metal,124,11.798287,Black Sabbath
3,14,R&B/Soul,53,5.042816,Amy Winehouse
4,6,Blues,36,3.425309,Buddy Guy
5,23,Alternative,35,3.330162,Chris Cornell
6,7,Latin,22,2.093245,Eric Clapton
7,9,Pop,22,2.093245,None
8,17,Hip Hop/Rap,20,1.902950,House Of Pain
9,2,Jazz,14,1.332065,Miles Davis


#### artist_name shows the artist who's tracks were most sold in a certain genre in USA.

#### Rock is most sold genre in USA and Van Halen is most sold rock artist in USA.

### total dollar amount of sales assigned to each sales support agent within the company

In [86]:
query = '''
with amount_paid as
(
select
inv.customer_id,
inv_l.unit_price * quantity as dollars_paid
from invoice_line as inv_l
LEFT JOIN invoice as inv
on inv_l.invoice_id = inv.invoice_id
),
agent_sales as
(
select customer.support_rep_id as agent_id,
SUM(amount_paid.dollars_paid) as amount_of_sales
from amount_paid
LEFT JOIN customer
on amount_paid.customer_id = customer.customer_id
GROUP BY 1
)
select employee.first_name || " " || employee.last_name as name,
employee.hire_date,
agent_sales.amount_of_sales
from agent_sales
INNER JOIN employee
on agent_sales.agent_id = employee.employee_id
'''
run_query(query)

,name,hire_date,amount_of_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


In [87]:
query = '''
with sales_customer as
(
select
inv.customer_id,
SUM(inv_l.unit_price * quantity) as dollars_spent
from invoice_line as inv_l
LEFT JOIN invoice as inv
on inv_l.invoice_id = inv.invoice_id
GROUP BY 1
),
country_sales as
(
select 
customer.country,
COUNT(customer.country) as no_of_customers,
sales_customer.dollars_spent as total_value_of_sales,
round(sales_customer.dollars_spent/cast(COUNT(customer.country) as Float), 2) as avg_sales_per_customer
from sales_customer
LEFT JOIN customer
ON sales_customer.customer_id = customer.customer_id
GROUP BY customer.country
)
select * from country_sales
ORDER BY 3 DESC
'''
run_query(query)

,country,no_of_customers,total_value_of_sales,avg_sales_per_customer
0,Czech Republic,2,128.70,64.35
1,Ireland,1,114.84,114.84
2,Brazil,5,106.92,21.38
3,Spain,1,98.01,98.01
4,Chile,1,97.02,97.02
5,Portugal,2,82.17,41.08
6,Australia,1,81.18,81.18
7,Finland,1,79.20,79.20
8,United Kingdom,3,79.20,26.40
9,Hungary,1,78.21,78.21


### a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
 - Number of invoices
 - Percentage of invoices


In [88]:
query = """
WITH invoice_info as
(
select invoice.invoice_id,
invoice.customer_id,
invoice_line.track_id
from invoice
LEFT JOIN invoice_line
on invoice.invoice_id = invoice_line.invoice_id
ORDER BY 1
),
track_info as
(
select track.track_id,
album.album_id
from track
INNER JOIN album
on track.album_id = album.album_id
ORDER BY 2
),
distinct_albums as
(
select invoice_info.invoice_id,
COUNT(track_info.track_id) as no_of_tracks,
COUNT(DISTINCT track_info.album_id) as no_of_albums
from invoice_info
LEFT JOIN track_info
on invoice_info.track_id = track_info.track_id
GROUP BY 1
),
same_or_different_album as
(
select 
CASE WHEN no_of_albums=1 
THEN 1
ELSE 0
END AS [same_album],
COUNT(invoice_id) as no_of_invoices
from distinct_albums
GROUP BY 1
)
select *,
CAST(no_of_invoices as FLOAT)/(select SUM(no_of_invoices) from same_or_different_album) as percentage
from same_or_different_album
"""
run_query(query)

,same_album,no_of_invoices,percentage
0,0,443,0.721498
1,1,171,0.278502


#### The store should stop buying full albums as we can see that almost 72 percent of the purchases were tracks from different albums and only 27 percent were from same album.

### How many tracks have been purchased vs not purchased?

In [89]:
query='''
WITH tracks as
(
select track_id
from track
),
tracks_sold as
(
select track_id as sold
from invoice_line
)
select COUNT(track_id) as no_of_tracks,
CASE WHEN sold IS NULL
THEN 0
ELSE 1
END AS [purchased]
from tracks
LEFT JOIN tracks_sold
on tracks.track_id = tracks_sold.sold
GROUP BY 2

'''
run_query(query)

,no_of_tracks,purchased
0,1697,0
1,4757,1


#### We see that almost 74 percent of the tracks have been purchased.